# Reranking Methods in RAG Systems

## 개요
재순위화는 검색 증강 생성(RAG) 시스템에서 검색된 문서의 관련성과 품질을 향상시키기 위한 중요한 단계입니다. 초기에 검색된 문서를 재평가하고 재정렬하여 후속 처리나 제시에 가장 관련성 높은 정보가 우선순위를 갖도록 합니다.

## 동기
RAG 시스템에서 재순위화를 사용하는 주요 동기는 종종 단순한 유사도 메트릭에 의존하는 초기 검색 방법의 한계를 극복하기 위함입니다. 재순위화는 더 정교한 관련성 평가를 가능하게 하며, 전통적인 검색 기법에서 놓칠 수 있는 쿼리와 문서 간의 미묘한 관계를 고려합니다. 이 과정은 생성 단계에서 가장 관련성 높은 정보가 사용되도록 보장하여 RAG 시스템의 전반적인 성능을 향상시키는 것을 목표로 합니다.

## 주요 구성 요소
재순위화 시스템은 일반적으로 다음 구성 요소를 포함합니다:

1. 초기 검색기: 종종 임베딩 기반 유사도 검색을 사용하는 벡터 저장소입니다.
2. 재순위화 모델: 다음 중 하나일 수 있습니다:
   - 관련성 점수를 매기기 위한 대규모 언어 모델(LLM)
   - 관련성 평가를 위해 특별히 훈련된 Cross-Encoder 모델
3. 점수 매기기 메커니즘: 문서에 관련성 점수를 할당하는 방법
4. 정렬 및 선택 로직: 새로운 점수를 기반으로 문서를 재정렬하기 위한 것

## 방법 상세
재순위화 과정은 일반적으로 다음 단계를 따릅니다:

1. 초기 검색: 잠재적으로 관련된 문서의 초기 집합을 가져옵니다.
2. 쌍 생성: 검색된 각 문서에 대해 쿼리-문서 쌍을 형성합니다.
3. 점수 매기기: 
   - LLM 방법: 프롬프트를 사용하여 LLM에 문서 관련성을 평가하도록 요청합니다.
   - Cross-Encoder 방법: 쿼리-문서 쌍을 모델에 직접 입력합니다.
4. 점수 해석: 관련성 점수를 파싱하고 정규화합니다.
5. 재정렬: 새로운 관련성 점수를 기반으로 문서를 정렬합니다.
6. 선택: 재정렬된 목록에서 상위 K개 문서를 선택합니다.

## 이 접근 방식의 장점
재순위화는 여러 이점을 제공합니다:

1. 관련성 향상: 더 정교한 모델을 사용함으로써 재순위화는 미묘한 관련성 요소를 포착할 수 있습니다.
2. 유연성: 특정 요구사항과 리소스에 따라 다른 재순위화 방법을 적용할 수 있습니다.
3. 컨텍스트 품질 향상: RAG 시스템에 더 관련성 높은 문서를 제공하면 생성된 응답의 품질이 향상됩니다.
4. 노이즈 감소: 재순위화는 관련성이 낮은 정보를 필터링하여 가장 관련성 높은 콘텐츠에 집중하는 데 도움이 됩니다.

## 결론
재순위화는 RAG 시스템에서 검색된 정보의 품질을 크게 향상시키는 강력한 기법입니다. LLM 기반 점수 매기기 또는 전문 Cross-Encoder 모델을 사용하든, 재순위화는 문서 관련성에 대한 더 미묘하고 정확한 평가를 가능하게 합니다. 이러한 향상된 관련성은 다운스트림 작업에서 더 나은 성능으로 직접 변환되어 재순위화를 고급 RAG 구현에서 필수 구성 요소로 만듭니다.

LLM 기반과 Cross-Encoder 재순위화 방법 중 선택은 필요한 정확도, 사용 가능한 계산 리소스, 특정 애플리케이션 요구사항과 같은 요소에 따라 달라집니다. 두 접근 방식 모두 기본 검색 방법에 비해 상당한 개선을 제공하며 RAG 시스템의 전반적인 효율성에 기여합니다.

<div style="text-align: center;">

<img src="../images/reranking-visualization.svg" alt="rerank llm" style="width:100%; height:auto;">
</div>

<div style="text-align: center;">

<img src="../images/reranking_comparison.svg" alt="rerank llm" style="width:100%; height:auto;">
</div>

# Package Installation and Imports

아래 셀은 이 노트북을 실행하는 데 필요한 모든 패키지를 설치합니다.


In [ ]:
# 필수 패키지 설치
#!pip install langchain langchain-openai python-dotenv sentence-transformers
!pip install -q faiss-cpu futures python-dotenv tqdm langchain langchain-community langchain-openai langchain-text-splitters langchain-core pypdf PyMuPDF pydantic>=2.0 rank-bm25 openai tiktoken deepeval

In [ ]:
# 헬퍼 함수 및 평가 모듈에 접근하기 위해 저장소 복제
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# 최신 데이터로 실행해야 하는 경우
# !cp -r RAG_TECHNIQUES/data .

In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain_core.documents import Document  # ✅ 수정됨
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain  # ✅ 수정됨
from langchain.chains.combine_documents import create_stuff_documents_chain  # ✅ 추가됨
from langchain_core.retrievers import BaseRetriever
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # ✅ 추가됨
from langchain_core.pydantic_v1 import BaseModel, Field  # ✅ 추가됨 (Pydantic 모델용)
from sentence_transformers import CrossEncoder

# Colab 호환성을 위해 원본 경로 추가 대체
from helper_functions import *
from evaluation.evalute_rag import *

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키 환경 변수 설정
if not os.getenv('OPENAI_API_KEY'):
    os.environ["OPENAI_API_KEY"] = input("Please enter your OpenAI API key: ")
else:
    os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')


### 문서 경로 정의

In [ ]:
# 필수 데이터 파일 다운로드
import os
os.makedirs('data', exist_ok=True)

# 이 노트북에서 사용되는 PDF 문서 다운로드
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [107]:
path = "data/Understanding_Climate_Change.pdf"

### 벡터 저장소 생성

In [108]:
vectorstore = encode_pdf(path)

## Method 1: LLM based function to rerank the retrieved documents

<div style="text-align: center;">

<img src="../images/rerank_llm.svg" alt="rerank llm" style="width:40%; height:auto;">
</div>

### 사용자 정의 재순위화 함수 생성


In [ ]:
# ✅ Pydantic 모델을 LangChain Core의 것으로 변경
from langchain_core.pydantic_v1 import BaseModel, Field

class RatingScore(BaseModel):
    relevance_score: float = Field(..., description="쿼리에 대한 문서의 관련성 점수.")

def rerank_documents(query: str, docs: List[Document], top_n: int = 3) -> List[Document]:
    prompt_template = PromptTemplate(
        input_variables=["query", "doc"],
        template="""On a scale of 1-10, rate the relevance of the following document to the query. Consider the specific context and intent of the query, not just keyword matches.
        Query: {query}
        Document: {doc}
        Relevance Score:"""
    )
    
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)
    llm_chain = prompt_template | llm.with_structured_output(RatingScore)
    
    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        score = llm_chain.invoke(input_data).relevance_score
        try:
            score = float(score)
        except ValueError:
            score = 0  # 파싱 실패 시 기본 점수
        scored_docs.append((doc, score))
    
    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]


### 문서와 관련된 샘플 쿼리로 재순위화 함수 사용 예제


In [ ]:
query = "기후 변화가 생물 다양성에 미치는 영향은 무엇인가요?"
initial_docs = vectorstore.similarity_search(query, k=15)
reranked_docs = rerank_documents(query, initial_docs)

# 처음 3개의 초기 문서 출력
print("초기 상위 문서:")
for i, doc in enumerate(initial_docs[:3]):
    print(f"\n문서 {i+1}:")
    print(doc.page_content[:200] + "...")  # 각 문서의 처음 200자 출력


# 결과 출력
print(f"쿼리: {query}\n")
print("재순위화된 상위 문서:")
for i, doc in enumerate(reranked_docs):
    print(f"\n문서 {i+1}:")
    print(doc.page_content[:200] + "...")  # 각 문서의 처음 200자 출력

### 재순위화 기반 사용자 정의 검색기 생성

In [ ]:
# 사용자 정의 검색기 클래스 생성
# ✅ LangChain 1.0 스타일로 수정
from langchain_core.pydantic_v1 import BaseModel as PydanticBaseModel

class CustomRetriever(BaseRetriever, PydanticBaseModel):
    
    vectorstore: Any = Field(description="초기 검색을 위한 벡터 저장소")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str, *, run_manager=None) -> List[Document]:
        """✅ get_relevant_documents 대신 _get_relevant_documents 사용"""
        initial_docs = self.vectorstore.similarity_search(query, k=30)
        return rerank_documents(query, initial_docs, top_n=2)

    async def _aget_relevant_documents(self, query: str, *, run_manager=None) -> List[Document]:
        """✅ 비동기 버전 추가"""
        raise NotImplementedError("비동기 검색이 구현되지 않았습니다")


# ✅ RetrievalQA 대신 create_retrieval_chain 사용

# 사용자 정의 검색기 생성
custom_retriever = CustomRetriever(vectorstore=vectorstore)

# 질문 답변을 위한 LLM 생성
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# ✅ 최신 방식: 프롬프트 템플릿 정의
system_prompt = """당신은 문서 기반 질의응답 AI입니다. 다음 문맥을 참고하여 정확하게 답변하세요.

Context: {context}

질문에 대한 답변을 작성하세요."""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# ✅ Document chain 생성
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# ✅ Retrieval chain 생성
qa_chain = create_retrieval_chain(custom_retriever, question_answer_chain)

# ✅ 기존: result = qa_chain({"query": query})
# ✅ 신규: result = qa_chain.invoke({"input": query})

query = "기후 변화가 생물 다양성에 미치는 영향은 무엇인가요?"
result = qa_chain.invoke({"input": query})

print(f"\n질문: {query}")
print(f"답변: {result['answer']}")  # ✅ 'result' 대신 'answer' 키 사용
print("\n관련 소스 문서:")
# ✅ 'source_documents' 대신 'context' 키 사용
for i, doc in enumerate(result["context"]):
    print(f"\n문서 {i+1}:")
    print(doc.page_content[:200] + "...")



### 예제 쿼리


In [ ]:
result = qa_chain({"query": query})

print(f"\n질문: {query}")
print(f"답변: {result['result']}")
print("\n관련 소스 문서:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n문서 {i+1}:")
    print(doc.page_content[:200] + "...")  # 각 문서의 처음 200자 출력

### 재순위화를 사용해야 하는 이유를 보여주는 예제 

In [ ]:
chunks = [
    "The capital of France is great.",
    "The capital of France is huge.",
    "The capital of France is beautiful.",
    """Have you ever visited Paris? It is a beautiful city where you can eat delicious food and see the Eiffel Tower. 
    I really enjoyed all the cities in france, but its capital with the Eiffel Tower is my favorite city.""", 
    "I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city."
]
docs = [Document(page_content=sentence) for sentence in chunks]


def compare_rag_techniques(query: str, docs: List[Document] = docs) -> None:
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(docs, embeddings)

    print("검색 기법 비교")
    print("==================================")
    print(f"쿼리: {query}\n")
    
    print("기준선 검색 결과:")
    baseline_docs = vectorstore.similarity_search(query, k=2)
    for i, doc in enumerate(baseline_docs):
        print(f"\n문서 {i+1}:")
        print(doc.page_content)

    print("\n고급 검색 결과:")
    custom_retriever = CustomRetriever(vectorstore=vectorstore)
    advanced_docs = custom_retriever.get_relevant_documents(query)
    for i, doc in enumerate(advanced_docs):
        print(f"\n문서 {i+1}:")
        print(doc.page_content)


query = "프랑스의 수도는 무엇인가요?"
compare_rag_techniques(query, docs)

Comparison of Retrieval Techniques
Query: what is the capital of france?

Baseline Retrieval Result:

Document 1:
The capital of France is great.

Document 2:
The capital of France is beautiful.

Advanced Retrieval Result:

Document 1:
I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city.

Document 2:
Have you ever visited Paris? It is a beautiful city where you can eat delicious food and see the Eiffel Tower. 
    I really enjoyed all the cities in france, but its capital with the Eiffel Tower is my favorite city.


## Method 2: Cross Encoder models

<div style="text-align: center;">

<img src="../images/rerank_cross_encoder.svg" alt="rerank cross encoder" style="width:40%; height:auto;">
</div>

### Cross Encoder 클래스 정의

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

class CrossEncoderRetriever(BaseRetriever, PydanticBaseModel):
    vectorstore: Any = Field(description="초기 검색을 위한 벡터 저장소")
    cross_encoder: Any = Field(description="재순위화를 위한 Cross-encoder 모델")
    k: int = Field(default=5, description="초기에 검색할 문서 수")
    rerank_top_k: int = Field(default=3, description="재순위화 후 반환할 문서 수")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str, *, run_manager=None) -> List[Document]:
        """✅ 메서드명 변경: get_relevant_documents -> _get_relevant_documents"""
        # 초기 검색
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)
        
        # Cross-encoder를 위한 쌍 준비
        pairs = [[query, doc.page_content] for doc in initial_docs]
        
        # Cross-encoder 점수 가져오기
        scores = self.cross_encoder.predict(pairs)
        
        # 점수별로 문서 정렬
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)
        
        # 상위 재순위화된 문서 반환
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

    async def _aget_relevant_documents(self, query: str, *, run_manager=None) -> List[Document]:
        """✅ 메서드명 변경"""
        raise NotImplementedError("비동기 검색이 구현되지 않았습니다")




### 인스턴스 생성 및 예제로 시연

In [ ]:
# Cross-encoder 검색기 생성
cross_encoder_retriever = CrossEncoderRetriever(
    vectorstore=vectorstore,
    cross_encoder=cross_encoder,
    k=10,
    rerank_top_k=5
)

# LLM 설정
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# ✅ 최신 방식으로 체인 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 문서 기반 질의응답 AI입니다. 다음 문맥을 참고하여 답변하세요:\n\n{context}"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
qa_chain = create_retrieval_chain(cross_encoder_retriever, question_answer_chain)

# ✅ 실행 방식 변경
query = "기후 변화가 생물 다양성에 미치는 영향은 무엇인가요?"
result = qa_chain.invoke({"input": query})

print(f"\n질문: {query}")
print(f"답변: {result['answer']}")
print("\n관련 소스 문서:")
for i, doc in enumerate(result["context"]):
    print(f"\n문서 {i+1}:")
    print(doc.page_content[:200] + "...")


![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--reranking)